In [ ]:
indicator_row_data = ''

In [ ]:
import re
import time

# Function to remove integer values between "~m~" and "~m~"
def remove_integers_from_payload(indicator_row_data):
    pattern = re.compile(r'~m~\d+~m~')
    item = pattern.sub('', indicator_row_data)
    return item

# Apply the function to the data
cleaned_data = remove_integers_from_payload(indicator_row_data)
cleaned_data

In [ ]:
import json
import re

def convert_payload_to_dict(payload):
    # Extract multiple JSON objects using regex
    json_objects = re.findall(r'\{.*?\}', payload)
    
    parsed_objects = []
    for obj in json_objects:
        try:
            parsed_objects.append(json.loads(obj))  # Convert each JSON object to a dictionary
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON: {e}")
            # print(obj)
            # print(type(obj))
    
    return parsed_objects

# Example usage:
converted_data = convert_payload_to_dict(cleaned_data)
converted_data 

In [ ]:
filtered_data = []

filtered_sublist = [item for item in converted_data if 'i' in item.keys() or 'v' in item.keys()]
filtered_data.append(filtered_sublist)

converted_data = filtered_data

In [ ]:
split_converted_data = []

for sublist in converted_data:

    unique_lengths = set()
    for item in sublist:
        print(item)
        unique_lengths.add(len(item['v']))

    unique_lengths = list(unique_lengths)
    unique_lengths.sort()

    # print(unique_lengths)

    # ------------------------------------------------

    length_dict = {}
    for item in sublist:
        length = len(item['v'])
        if length not in length_dict:
            length_dict[length] = []
        length_dict[length].append(item)

    # print(length_dict)
    
    for length, items in length_dict.items():
        split_converted_data.append(items)

# split_converted_data

In [ ]:
import pandas as pd

df_list = []

for a in range(0, len(split_converted_data)):

    qw = 'df' + str(a)

    df_list.append(qw)

for i in range(0, len(split_converted_data)):

    # List of dictionaries
    data_list = split_converted_data[i]

    # Convert to DataFrame
    df = pd.DataFrame(data_list)

    # Expand the 'v' column into separate columns
    df_expanded = pd.DataFrame(df['v'].tolist(), index=df['i'])

    length_list = ['timestamp']

    for q in range(0, len(split_converted_data[i][0]['v'])):

        if q != 0:
            length_list.append(str(q))

    print(length_list)

    # Rename columns
    df_expanded.columns = length_list

    df_expanded['timestamp'] = pd.to_datetime(df_expanded['timestamp'], unit='s', utc=True)
    # df_expanded['timestamp'] = df_expanded['timestamp'].dt.tz_convert('Asia/Kolkata')
    df_expanded['timestamp'] = df_expanded['timestamp'].dt.tz_localize(None)
    df_expanded.rename(columns={'timestamp': 'datetime'}, inplace=True)

    # Print the updated DataFrame
    df_expanded = df_expanded.reset_index(drop=True)

    print(i)

    df_list[i] = df_expanded

    # Print the DataFrame
    # qw = 'df' + str(i)

In [ ]:
df = pd.concat(df_list, ignore_index=True)
df

In [ ]:
df.head(1)

In [ ]:
# columns_to_remove = ['2', '4', '6', '8']
# df = df.drop(columns=columns_to_remove)
# df

In [ ]:
df.rename(columns={'1': 'rising_', '2': 'falling_', '3': 'c_', '4': 'hi_', '5': 'lo_', 
                    '6': 'c1_', '7': 'anchoredHi_', '8': 'anchoredLo_', '9': 'h1_', '10': 'l1_', 
                    '11': 'o_', '12': 'doji_', '13': 'breakingHigher_', '14': 'breakingLower_', '15': 'resetHiAnchor_', 
                    '16': 'resetLoAnchor_', '17': 'VWAP', '18': 'Anchor_Hi', '19': 'Anchor_Lo', '20': 'anchoredMean'}, inplace=True)
df

In [ ]:
replace_dict = {
    1.0: True, 11.0: True, 21.0: True, 31.0: True, 41.0: True, 51.0: True, 61.0: True,
    2.0: False, 12.0: False, 22.0: False, 32.0: False, 42.0: False, 52.0: False, 62.0: False
}

columns_to_replace = ['rising_', 'falling_', 'doji_', 'breakingHigher_', 'breakingLower_', 'resetHiAnchor_', 'resetLoAnchor_']

df[columns_to_replace] = df[columns_to_replace].replace(replace_dict)

In [ ]:
stock_name = input('Enter the stock name: ')
df.to_csv(f'{stock_name}_AutoHI_Lo.csv', index=False)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
